In [ ]:
from tensorflow.python import train
import math
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# download the required dataset from yaho stock prices
company = 'AAPL'
stock_data  = yf.download(company,start='2018-01-01', end='2023-01-01')
stock_data.head()

#visualize the stock prices
# plt.figure(figsize=(15,8))
# plt.title("Stock Prices History")
# plt.plot(stock_data['Close'])
# plt.xlabel('Date')
# plt.ylabel('Prices ($)')


#preparing the training dataset
close_prices = stock_data['Close']
values  = close_prices.values #convert to arrays
training_data_len = math.ceil(len(values)*0.8) #extract 80% of data

scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(values.reshape(-1,1)) # normalize data to 2d array in range 0,1 
train_data = scaled_data[0: training_data_len,:] # extract 80% data as training data

x_train = [] 
y_train = []

for i in range(60,len(train_data)):
    x_train.append(train_data[i-60:i, 0]) #create 60 day window of prices
    y_train.append(train_data[i, 0])

x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))    #reshape into 3d array 


# Preparing the test data
test_data = scaled_data[training_data_len-60: , : ] # last 20% of data
x_test = []
y_test = values[training_data_len:]

for i in range(60, len(test_data)):
  x_test.append(test_data[i-60:i, 0])

x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))


# Setting up an lstm network

model = keras.Sequential()

#LSTM layer by giving it 100 network units. Set the return_sequence to true so that the output of the layer will be another sequence of the same length.
model.add(layers.LSTM(100, return_sequences=True, input_shape=(x_train.shape[1], 1)))

#Add another LSTM layer with also 100 network units. But we set the return_sequence to false for this time to only return the last output in the output sequence.
model.add(layers.LSTM(100, return_sequences=False))

#add a densely connected neural network layer with 25 network units.
model.add(layers.Dense(25))

model.add(layers.Dense(1))
model.summary()



# Training the LSTM model
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, batch_size=1, epochs=3)



# Evaluate the model
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions) # denormalize the predicted stock prices

# RMSE formula to calculate the degree of discrepancy between the predicted prices and real prices
rmse = np.sqrt(np.mean(predictions - y_test)**2)
rmse


# Visualizing the predicted prices
data = stock_data.filter(['Close'])
train = data[:training_data_len]
validation = data[training_data_len:]
validation['Predictions'] = predictions
plt.figure(figsize=(16,8))
plt.title('Model')
plt.xlabel('Date')
plt.ylabel('Close price USD ($)')
plt.plot(train)
plt.plot(validation[['Close', 'Predictions']])
plt.legend(['Train', 'Val', 'Predictions'], loc='lower right')
plt.show()

